# Blip2 COCO Captions

In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from datasets import COCODataset
from datasets import COCODataset
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader

# Load COCO dataset
coco_dataset = COCODataset(
    ann_file="./data/coco/annotations/captions_val2017.json",
    img_dir="./data/coco/val2017",
)

In [ ]:
len(coco_dataset)

## Collect Inference Results

In [ ]:
import json


def eval_model(qmodel, results_file="./results/inference.json", eval_size=50):
    if results_file == "./results/inference.json":
        print('No results file provided, using default "./results/inference.json"')

    results = []

    for i in tqdm(range(0, min(eval_size, len(coco_dataset)))):
        image, _ = coco_dataset[i]

        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            out = qmodel.generate(**inputs)

        caption = processor.decode(out[0], skip_special_tokens=True).strip()

        image_id = coco_dataset.ids[i]
        results.append({"image_id": image_id, "caption": caption})

    with open(results_file, "w") as f:
        json.dump(results, f)

In [ ]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.cider.cider import Cider
import sys
import os

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)


class SimpleCIDErEval:
    def __init__(self):
        self.tokenizer = PTBTokenizer()
        self.cider_scorer = Cider()

    def evaluate(self, predictions, references):
        # Format the input for the tokenizer
        gts = {i: [{"caption": c} for c in refs] for i, refs in enumerate(references)}
        res = {i: [{"caption": p}] for i, p in enumerate(predictions)}

        # Tokenize
        gts_tokenized = self.tokenizer.tokenize(gts)
        res_tokenized = self.tokenizer.tokenize(res)

        # Compute CIDEr score
        score, scores = self.cider_scorer.compute_score(gts_tokenized, res_tokenized)

        return score, scores


def eval_results(results_file):
    f = open(results_file)
    results = json.load(f)
    f.close()

    candidates = [result["caption"] for result in results]
    references = [coco_dataset.get_captions(result["image_id"]) for result in results]

    evaluator = SimpleCIDErEval()

    overall_score, individual_scores = evaluator.evaluate(candidates, references)

    return overall_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import copy
import time


def quant(x: torch.Tensor, num_bits):
    min_val = x.min()
    max_val = x.max()

    if min_val == max_val:
        return x

    scale = (2**num_bits - 1) / (max_val - min_val)
    zero_point = (-min_val * scale).round()

    x_int = (x * scale + zero_point).round().clamp(0, 2**num_bits - 1)
    x_quant = (x_int - zero_point) / scale

    return x_quant


def replace_linear_with_quantized(model: nn.Module, weight_bits: int = 2, dynamic=False) -> nn.Module:
    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            module.weight.data = quant(module.weight.data, weight_bits)
            if module.bias is not None:
                module.bias.data = quant(module.bias.data, weight_bits)
        elif isinstance(module, nn.Module):
            replace_linear_with_quantized(module, weight_bits, dynamic)
    return model


bit_sizes = [16, 8, 6, 4, 2]
eval_size = 1000
scores = {}

model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b")
model = model.to(device)
print("Evaluating Base Model")
results_file = f"./results/quant_32.json"
eval_model(model, results_file, eval_size)
print("Finished Evaluating")
score = eval_results(results_file)
scores[32] = score
print(f"Score: {score}")
del model
torch.cuda.empty_cache()
time.sleep(2)

for i, bits in enumerate(bit_sizes):
    model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b")
    model = model.to(device)

    print(f"\nReplacing linear layers with weight bits:{bits}")
    model = replace_linear_with_quantized(model, weight_bits=bits, dynamic=False)
    print("Evaluating Model")
    results_file = f"./results/quant_{bits}.json"
    eval_model(model, results_file, eval_size)
    print("Finished Evaluating")
    score = eval_results(results_file)
    scores[bits] = score

    print(f"Score: {score}")
    del model
    torch.cuda.empty_cache()
    time.sleep(2)

In [ ]:
scores

In [ ]:
import matplotlib.pyplot as plt

keys = list(scores.keys())
values = list(scores.values())

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(keys, values, marker="o")

# Customize the plot
plt.title("Scores Plot")
plt.xlabel("Key")
plt.ylabel("Score")
plt.grid(True)

# Set x-axis ticks to only show the actual key values
plt.xticks(keys)

# Add value labels
for i, v in enumerate(values):
    plt.text(keys[i], v, f"{v:.4f}", ha="center", va="bottom")

# Adjust y-axis to start from 0
plt.ylim(bottom=0)

# Show the plot
plt.show()

In [ ]:
eval_size = 1000
scores = {}

model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b")
model = model.to(device)
print("Evaluating Base Model")
results_file = f"./results/quant_32.json"
eval_model(model, results_file, eval_size)
print("Finished Evaluating")
score = eval_results(results_file)
scores[32] = score
print(f"Score: {score}")
del model
torch.cuda.empty_cache()

In [ ]:
import torch
import numpy as np


def quant1(x: torch.Tensor, num_bits):
    min_val = x.min()
    max_val = x.max()

    if min_val == max_val:
        return x

    scale = (2**num_bits - 1) / (max_val - min_val)
    zero_point = (-min_val * scale).round()

    x_int = (x * scale + zero_point).round().clamp(0, 2**num_bits - 1)
    x_quant = (x_int - zero_point) / scale

    return x_quant


def quant2(x: torch.Tensor, num_bits):
    min_val = x.min(dim=-1).values.unsqueeze(-1)
    max_val = x.max(dim=-1).values.unsqueeze(-1)

    alpha = max_val - min_val

    x = (x - min_val) / alpha

    scale = 2**num_bits - 1

    result = (scale * x).round()

    result /= scale

    result = alpha * result + min_val

    return result


def compare_quant_functions(num_samples=1000, tensor_size=100, num_bits=8):
    max_diff = 0
    avg_diff = 0

    for _ in range(num_samples):
        # Generate random tensor
        x = torch.randn(tensor_size)

        # Apply both quantization functions
        q1 = quant1(x, num_bits)
        q2 = quant2(x, num_bits)

        # Calculate difference
        diff = torch.abs(q1 - q2)
        max_diff = max(max_diff, diff.max().item())
        avg_diff += diff.mean().item()

    avg_diff /= num_samples

    print(f"Maximum difference: {max_diff}")
    print(f"Average difference: {avg_diff}")


# Run the comparison
compare_quant_functions()